In [64]:
import cv2
from ultralytics import YOLO

In [65]:
# 載入模型
model = YOLO('yolov8s.pt')

# 讀取影片檔案
cap = cv2.VideoCapture('argoverse.mp4')

In [66]:
# 確認影片成功打開
if not cap.isOpened():
    print("Error: File can not be opened")
    exit()

In [67]:
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
# print("width:",frame_width, "height:", frame_height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

width: 1920 height: 1200


In [68]:
while True:
    # 讀取影片的每一幀
    ret, frame = cap.read()
    
    # 如果已經讀取到影片結尾，則退出循環
    if not ret:
        print("End")
        break
    
    # 使用 YOLOv8 模型進行物件偵測並繪製偵測框和標籤
    results = model.predict(frame, verbose = False)
    annotated_frame = frame.copy()
    
    for result in results:
        for box in result.boxes:
            # 取得類別名稱和置信度
            cls = box.cls
            conf = float(box.conf)
            name = model.names[int(cls)]
            
            # 篩選出你想辨識的特定物件（例如 'car'）
            if name == 'car':
                # 取得偵測框的位置
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # 繪製偵測框
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # 繪製標籤
                label = f"{name} {conf:.2f}"
                cv2.putText(annotated_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


    # 顯示
    # cv2.imshow('Video', frame)
    # cv2.imshow('a', annotated_frame)
    # height, width = annotated_frame.shape[:2]
    # print("width:",width, "height:", height)
    out.write(annotated_frame)
    # 按下 'q' 鍵退出視窗，或按下 'Esc' 鍵退出
    key = cv2.waitKey(25) & 0xFF
    if key == ord('q') or key == 27:  # 27 是 Esc 鍵的 ASCII 碼
        break

End


In [69]:
# 釋放影片捕捉物件並關閉所有 OpenCV 視窗
cap.release()
out.release()
cv2.destroyAllWindows()